In [1]:
MODEL_NAME = "mistralai/Mistral-7B-v0.1"


This is about laying out the steps an LLM goes through to generate a response when it is fed a prompt.
We assume some background on LLMs, but you don't need to be an engineer / know how to code to follow along.

With that said you can copy and paste any of the code into ChatGPT and ask it to "explain what this code does in plain English" and it should do a solid job!

## Converting Words into Tokens
Key Terms:
"tokens"
"embeddings"
"matrices"

At their core, what machine learning models do is take in an input and predict what the most likely output is. W

The tokenizer plays the key role of converting language understandable by humans (e.g. English), into the inputs that are understandable by the model (i.e. numbers, and ultimately 1s and 0s).

Each model has its own accompanying Tokenizer that is trained alongside it, you can't think of it as a special converter that turns the language into this specific language

NOTE NEED SOME NICE METAPHOR

We'll start by loading a tokenizer from HuggingFace...



In [2]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The prompt we want to use is "Can you write an email explaining what an LLM is?" we'll start by defining this sentence for the model and giving it to the tokenizer to process...

In [3]:
PROMPT = "Can you write an email explaining what an LLM is?"
inputs = tokenizer(PROMPT, return_tensors="pt")

The model breaks down the prompt into sub-word chunks called "tokens". What Large Language Models (LLMs) are trained to do is take in this string of "tokens", and perform "next token prediction" (i.e. guess the next token) based on the examples it has been "trained on" in the past.

In [4]:
import pandas as pd
from tabulate import tabulate


tokens_and_ids = []
for i, token_id in enumerate(inputs["input_ids"][0]):
    token_text = tokenizer.decode([token_id])
    tokens_and_ids.append((token_text, token_id.item()))


df = pd.DataFrame(tokens_and_ids, columns=['Token Text', 'Token ID'])
df


,Token Text,Token ID
0,<s>,1
1,Can,2418
2,you,368
3,write,3324
4,an,396
5,email,4927
6,explaining,20400
7,what,767
8,an,396
9,LL,16704


The tokenizer knows which Token ID corresponds to what Token (i.e."sub-word chunk of text"). The Token IDs are the numbers that the model actually uses to process the text.

I am onlying showing the Token Text for explanation purposes!


*Side Note: You'll also notice that the list of tokens begins with "< s >" which is the indicator to the model that a prompt or response is starting.


## Loading the Model

Here I want to actually load the model

In [23]:
import torch

torch.cuda.empty_cache()

In [25]:
!nvidia-smi -q -d Memory | grep -A4 GPU

Attached GPUs                             : 1
GPU 00000000:00:04.0
    FB Memory Usage
        Total                             : 16384 MiB
        Reserved                          : 232 MiB
        Used                              : 16076 MiB


In [24]:
!nvidia-smi

Mon Feb 19 21:43:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              41W / 300W |  16076MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
model = AutoModel.from_pretrained(MODEL_NAME)

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacty of 15.77 GiB of which 74.38 MiB is free. Process 88467 has 15.70 GiB memory in use. Of the allocated memory 15.21 GiB is allocated by PyTorch, and 137.42 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

And then here I move the model to the memory on a Nvidia GPU attached to my computer. You can think of this almost like copy and pasting

In [8]:
model

NameError: name 'model' is not defined